The dataset used in this notebook can be found at:
https://www.kaggle.com/datasets/samithsachidanandan/human-face-emotions

In [ ]:
from torchvision import transforms
from torchvision.datasets import ImageFolder
import torch
import torch.optim as optim
from torch.utils.data import Subset
from skorch import NeuralNetClassifier
from skorch.callbacks import EarlyStopping

import seaborn as sns
from itertools import islice
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from pathlib import Path

In [ ]:
IMG_SIZE = 64
DATASET_PATH = Path("../data/Emotions/")

FIGURES_DIR = Path("figures/emotions/")
FIGURES_DIR.mkdir(parents=True, exist_ok=True)

In [ ]:
from computer_vision.src.transforms import get_transform

transform = get_transform(IMG_SIZE, [0.5117, 0.5098, 0.5089], [0.2070, 0.2062, 0.2060])

# the above mean and std were computed from the training set with the following function:


ds = ImageFolder(root=DATASET_PATH,
                 transform=transform)

In [ ]:
TEST_SIZE = 0.2
SEED = 42

# generate indices: instead of the actual data we pass in integers instead
train_indices, test_indices, _, _ = train_test_split(
    range(len(ds)),
    ds.targets,
    stratify=ds.targets,
    test_size=TEST_SIZE,
    random_state=SEED
)

# generate subset based on indices
train_dataset = Subset(ds, train_indices)
test_dataset = Subset(ds, test_indices)


In [ ]:
len(test_dataset), len(train_dataset)

# Visualize some images

In [ ]:
X_example, y_example = zip(*islice(iter(train_dataset), 7))

In [ ]:
from computer_vision.src.figures import plot_example

plot_example(torch.stack(X_example), y_example, ds.classes, n=7, mean=transform.transforms[2].mean, std=transform.transforms[2].std);

# Distribution of classes in the training set

In [ ]:
from computer_vision.src.figures import plot_label_distribution

figures_name = FIGURES_DIR / "class_distribution.png"

plot_label_distribution(ds, figures_name)

# Training a baseline model

In [ ]:
y_train = np.array([y for x, y in iter(train_dataset)])
y_test = np.array([y for x, y in iter(test_dataset)])

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
from computer_vision.src.baseline import BaselineModel
torch.manual_seed(0)

baseline = NeuralNetClassifier(
    BaselineModel,
    max_epochs=10,
    iterator_train__num_workers=2,
    iterator_valid__num_workers=2,
    lr=0.1,
    device=device,
    callbacks=[EarlyStopping(patience=3)],
    module__output_dim=5,
    module__input_dim=IMG_SIZE*IMG_SIZE*3,
)

In [ ]:
baseline.fit(train_dataset, y=y_train);

## Plot training loss

In [ ]:
train_loss_history = baseline.history[:, 'train_loss']
val_loss_history = baseline.history[:, 'valid_loss']
sns.lineplot(x=range(1, len(train_loss_history) + 1), y=train_loss_history, label='Train Loss')
sns.lineplot(x=range(1, len(val_loss_history) + 1), y=val_loss_history , label='Validation Loss')
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Training Loss over Epochs")
plt.show()

## Evaluating the model

In [ ]:
print("Test set Accuracy: {:.2f}%".format( 100 * baseline.score(test_dataset, y_test)))

# Use BaseCNN with custom parameters

In [ ]:
from computer_vision.src.BaseCNN import BaseCNN
from sklearn.model_selection import GridSearchCV
from skorch.helper import SliceDataset


cnn = NeuralNetClassifier(
    BaseCNN,
    max_epochs=10,
    lr=0.001,
    optimizer=optim.Adam,
    device=device,
    callbacks=[EarlyStopping(patience=3)],

    module__num_classes=5,
    module__img_size=IMG_SIZE,
    module__nb_conv_layers=2,
    module__nb_layers=2,
    module__net_width=256,
    module__dropout_rates=[0.25, 0.5],
)

In [ ]:
cnn.fit(train_dataset, y_train)

In [ ]:
train_loss_history = cnn.history[:, 'train_loss']
valid_loss_history = cnn.history[:, 'valid_loss']

sns.lineplot(x=range(1, len(train_loss_history) + 1), y=train_loss_history, label='Train Loss')
sns.lineplot(x=range(1, len(valid_loss_history) + 1), y=valid_loss_history , label='Validation Loss')
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Training Loss over Epochs")
plt.show()

In [ ]:
print("Test set Accuracy: {:.2f}%".format(
    100 * cnn.score(test_dataset, y_test)
))

# Hyperparameter Tuning with Grid Search

In [ ]:
net = NeuralNetClassifier(
    BaseCNN,
    max_epochs=10,
    lr=0.1,
    train_split=False,
    callbacks=[EarlyStopping(patience=5, monitor='train_loss')],

    module__num_classes=5,
    module__img_size=IMG_SIZE,
    device=device,

)

In [ ]:
params = {
    #'lr': [0.01, 0.02],
    #'max_epochs': [10, 20],
    #'batch_size': [32, 64],
    #'optimizer': [optim.Adam, optim.SGD],
    'module__nb_conv_layers':[2, 3],
    'module__nb_layers':[2, 3],
    'module__net_width':[128, 256],
    'module__dropout_rates':[[0],[0.25, 0.5]],
}

In [ ]:
grid = GridSearchCV(net, params, cv=3, scoring='accuracy', verbose=2, n_jobs=-1)

In [ ]:
train_dataset_sliceable = SliceDataset(train_dataset)

In [ ]:
grid.fit(train_dataset_sliceable, y_train)

In [ ]:
print("Best parameters found: ", grid.best_params_)
print("Best cross-validation accuracy: ", grid.best_score_)

In [ ]:
print("Test set accuracy: ", grid.score(SliceDataset(test_dataset), y_test))